In [8]:
import tensorflow as tf
import pandas as pd

In [9]:
df = pd.read_csv('refined_used_car.csv')
LABEL_COLUMN = 'Price'

In [10]:
def features_and_labels(row_data):
    """Splits features and labels from feature dictionary.

    Args:
        row_data: Dictionary of CSV column names and tensor values.
    Returns:
        Dictionary of feature tensors and label tensor.
    """
    label = row_data.pop(LABEL_COLUMN)

    return row_data, label  # features, label


def load_dataset(pattern, batch_size=1, mode=tf.estimator.ModeKeys.EVAL):
    """Loads dataset using the tf.data API from CSV files.

    Args:
        pattern: str, file pattern to glob into list of files.
        batch_size: int, the number of examples per batch.
        mode: tf.estimator.ModeKeys to determine if training or evaluating.
    Returns:
        `Dataset` object.
    """
    # TODO: Make a CSV dataset
    dataset = tf.data.experimental.make_csv_dataset(pattern, batch_size) #, DEFAULTS)

    # TODO: Map dataset to features and label
    dataset = dataset.map(features_and_labels)  # features, label

    # Shuffle and repeat for training
    if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.shuffle(buffer_size=1000).repeat()

    # Take advantage of multi-threading; 1=AUTOTUNE
    dataset = dataset.prefetch(buffer_size=1)

    return dataset

In [11]:
NUMERIC_COLUMNS = [
 'Mileage',
 'Mpg',
]

CATEGORICAL_COLUMNS = [
 'Transmission',
 'FuelType',
 'Year',
 'EngineSize',
]

In [12]:
CATEGORIES = {
    'Transmission': ['Automatic', 'Manual', 'Semi-Auto', 'Other'], 'FuelType': ['Petrol', 'Hybrid', 'Diesel', 'Other', 'Electric'], 
    'Year': [2005, 2017, 2016, 2011, 2018, 2012, 2019, 2020, 2014, 2015, 2006, 2010, 2004, 2008, 2013, 2007, 2009, 2003, 2001, 2002, 1998, 2000, 1997, 1999, 1996, 1991, 1995], 
    'EngineSize': [1.8, 2.1, 5.5, 4.0, 6.2, 3.5, 2.0, 1.5, 3.0, 1.2, 1.6, 1.4, 1.7, 2.5, 4.7, 1.3, 2.2, 2.9, 0.6, 2.3, 0.8, 1.0, 6.0, 3.2, 4.4, 5.0, 2.7, 3.7, 5.4, 2.8, 1.1, 1.9, 2.4, 4.2, 4.5, 3.6, 2.6, 6.6, 5.2, 4.1, 6.3]
}

In [13]:
def get_categorical(name, voc_list):
    cat = tf.feature_column.categorical_column_with_vocabulary_list(name, voc_list)
    return tf.feature_column.indicator_column(cat)

def create_feature_columns(nembeds):
    """Creates wide and deep dictionaries of feature columns from inputs.

    Args:
        nembeds: int, number of dimensions to embed categorical column down to.
    Returns:
        Wide and deep dictionaries of feature columns.
    """
    # TODO: Create deep feature columns for numeric features
    deep_fc = {
        colname: tf.feature_column.numeric_column(colname) for colname in NUMERIC_COLUMNS
    }

    # TODO: Create wide feature columns for categorical features
    wide_fc = {
        colname: get_categorical(colname, list(CATEGORIES[colname])) for colname in CATEGORICAL_COLUMNS
    }

#     # TODO: Bucketize the float fields. This makes them wide
#     age_bucket = tf.feature_column.bucketized_column(source_column=deep_fc['mother_age'],
#                                                     boundaries=np.arange(15, 45, 1).tolist())
#     wide_fc['age_bucket'] = tf.feature_column.indicator_column(age_bucket)

#     gestation_bucket = tf.feature_column.bucketized_column(source_column=deep_fc['gestation_weeks'],
#                                                     boundaries=np.arange(17, 47, 1).tolist())
#     wide_fc['gestation_bucket'] = tf.feature_column.indicator_column(gestation_bucket)

#     # TODO: Cross all the wide cols, have to do the crossing before we one-hot
#     crossed = tf.feature_column.crossed_column([age_bucket, gestation_bucket], 1000)

#     # TODO: Embed cross and add to deep feature columns
#     deep_fc['crossed'] = tf.feature_column.embedding_column(crossed, dimension=nembeds)

    return wide_fc, deep_fc

In [14]:
create_feature_columns(1)

ValueError: column_name: EngineSize vocabulary dtype must be string or integer. dtype: <dtype: 'float64'>.